In [1]:
# Integrating with ganache

In [2]:
# Importing libraries
from web3 import Web3
import json

In [3]:
#Connection to Blockchain
from web3 import Web3
web3 = Web3(Web3.HTTPProvider('HTTP://127.0.0.1:7545'))
print("isConnected:",web3.isConnected())

#gettin lists of accounts available, Account[0] will be like an auditor account
accounts_list = web3.eth.accounts
for i in accounts_list:
    print(i)

isConnected: True
0x2c2cc39185c81247F13FCd9d221EedBE5B3cA67E
0xc1a28A333D77eE25eA6dcB96DCe51Db6a8a78E75
0x27c4FA9f17e939203F2986660ad5C87b4C62c426
0x72280e1c8BbeD3d4472fad1a6e2FFBb5B0E89835
0xB59F6Cb8fbE4133009339490bB037e595Fc7C666



### Initial setup(s).
1. First account in the list will be deployer of the contracts:<br>
    1.1. `account[0]` owner of contract `Manufacturer`<br>
    1.2. `account[0]` owner of contract `Consumer`
<br>
2. Rest other accounts will be consumers. <i><u>Note:</u></i> at the moment manufacturere is also or can act as a consumer.


In [4]:
#storing variables of manufacture
manufacturer_address = accounts_list[0]
print("Medicine Manufacturer:",manufacturer_address)


Medicine Manufacturer: 0x2c2cc39185c81247F13FCd9d221EedBE5B3cA67E


In [5]:
#Function to createPill
def createPill(owner): #onlyManufacturer
    try:
        transaction_hash = contract_manufacturer.functions.createpills().transact({'from':owner})
        return transaction_hash.hex()
    
    except ValueError:
        return "Error: Only manufacturere can make pills"
    
#adding function to consume pill
    
        
#Function to check pillBook
def pillBookData(pillIndex):
    fields = ['Description','strength(mg)','imprints','pregnancyCategory','csaSchedule','consumedFalse','consumedBy']
    _ = contract_manufacturer.functions.pillBook(pillIndex).call()
    #zipping both:
    return dict(zip(fields,_))

#### Deploying `Manufacturer` contract 

In [7]:
# importing contract_address and abi from json
with open('build/contracts/Manufacturer.json') as f:
    manufacturer = json.load(f)

In [8]:
manufacturer_contract_address = manufacturer['networks']['5777']['address']
manufacturer_contract_abi = manufacturer['abi']
contract_manufacturer = web3.eth.contract(address = manufacturer_contract_address, abi = manufacturer_contract_abi)

#### Playing with Contract

In [9]:
#1. Creating 5 pills
#Only manufacturer can manufacture pills
print(createPill(manufacturer_address))
print(createPill(manufacturer_address))
print(createPill(manufacturer_address))
print(createPill(manufacturer_address))
print(createPill(manufacturer_address))


0xf075fb38125c8816fa8e4850ffa046a1e221ff4a39ae50208a3dd8c127037485
0xbc199807b4511552a39d43b6e481dc20089bfe56d3e228c6141e37290e343960
0x36e070339fa6a6dc27cde9cbf5817d45b3a2c98c4bec63b96c1d0f82c69e84b3
0x6cce8e81e2b4f4c513299714c1f9384bcf42dc7d3461f4cd21797c78c6303198
0xb88b2734139b23444fd6bb2816fe764d31b09c95d9c6c1f22c2a799c3437809a


In [10]:
# Getting State Variables
# Company Name
print('Company Name:',contract_manufacturer.functions.company().call())
# Contract Owner
print('Owner of Manufacturer Contact (address)',contract_manufacturer.functions.contractOwner().call())
# Production sequece, i.e. checking howmany produced from 1 to N-1
print('Next production sequence Number:',contract_manufacturer.functions.produnctionSequece().call())

Company Name: PharmaCompany
Owner of Manufacturer Contact (address) 0x2c2cc39185c81247F13FCd9d221EedBE5B3cA67E
Next production sequence Number: 6


#### Deploying `Consumer` contract 

In [11]:
with open('build/contracts/Consumer.json') as f:
    consumer = json.load(f)

In [12]:
consumer_contract_address = consumer['networks']['5777']['address']
consumer_contract_abi = consumer['abi']
contract_consumer = web3.eth.contract(address = consumer_contract_address, abi = consumer_contract_abi)

In [13]:
#KECCAK of int(1->5):
#1. 0xb10e2d527612073b26eecdfd717e6a320cf44b4afac2b0732d9fcbe2b7fa0cf6
#2. 0x405787fa12a823e0f2b7631cc41b3ba8828b3321ca811111fa75cd3aa3bb5ace
#3. 0xc2575a0e9e593c00f959f8c92f12db2869c3395a3b0502d05e2516446f71f85b
#4. 0x8a35acfbc15ff81a39ae7d344fd709f28e8600b4aa8c65c6b64bfe7fe36bd19b
#5. 0x036b6384b5eca791c62761152d0c79bb0604c104a5fb6f4eb0703f3154bb3db0

In [14]:
#0x3a3c79c8993ceac4696e115af8b9ba2dfe605009910e21eccd6523469e3c4b99
_ = pillBookData('0xb10e2d527612073b26eecdfd717e6a320cf44b4afac2b0732d9fcbe2b7fa0cf6')
print(_)

{'Description': 1, 'strength(mg)': 10, 'imprints': 'PD 155 10', 'pregnancyCategory': 4, 'csaSchedule': 6, 'consumedFalse': False, 'consumedBy': '0x0000000000000000000000000000000000000000'}


In [15]:
#Now consumer can only consume,
#assuming that he is not call function to call information about the pill.
#consumption we are doing only from other contract.

#consuming pill with correct indexes
pill_index = 1

pill_consumption_hash = contract_consumer.functions.consumeThePill(pill_index).transact({'from':accounts_list[1],'value': 1000000000000000000})
print(pillBookData('0xb10e2d527612073b26eecdfd717e6a320cf44b4afac2b0732d9fcbe2b7fa0cf6'))

{'Description': 1, 'strength(mg)': 10, 'imprints': 'PD 155 10', 'pregnancyCategory': 4, 'csaSchedule': 6, 'consumedFalse': True, 'consumedBy': '0xc1a28A333D77eE25eA6dcB96DCe51Db6a8a78E75'}


In [17]:
pill_index = 2

pill_consumption_hash = contract_consumer.functions.consumeThePill(pill_index).transact({'from':accounts_list[2],'value': 1000000000000000000})
print(pillBookData('0x405787fa12a823e0f2b7631cc41b3ba8828b3321ca811111fa75cd3aa3bb5ace'))

{'Description': 2, 'strength(mg)': 10, 'imprints': 'PD 155 10', 'pregnancyCategory': 4, 'csaSchedule': 6, 'consumedFalse': True, 'consumedBy': '0x27c4FA9f17e939203F2986660ad5C87b4C62c426'}


In [16]:
pill_index = 3

pill_consumption_hash = contract_consumer.functions.consumeThePill(pill_index).transact({'from':accounts_list[3],'value': 1000000000000000000})
print(pillBookData('0xc2575a0e9e593c00f959f8c92f12db2869c3395a3b0502d05e2516446f71f85b'))

{'Description': 3, 'strength(mg)': 10, 'imprints': 'PD 155 10', 'pregnancyCategory': 4, 'csaSchedule': 6, 'consumedFalse': True, 'consumedBy': '0x72280e1c8BbeD3d4472fad1a6e2FFBb5B0E89835'}


In [18]:
pill_index = 4

pill_consumption_hash = contract_consumer.functions.consumeThePill(pill_index).transact({'from':accounts_list[3],'value': 1000000000000000000})
print(pillBookData('0x8a35acfbc15ff81a39ae7d344fd709f28e8600b4aa8c65c6b64bfe7fe36bd19b'))

{'Description': 4, 'strength(mg)': 10, 'imprints': 'PD 155 10', 'pregnancyCategory': 4, 'csaSchedule': 6, 'consumedFalse': True, 'consumedBy': '0x72280e1c8BbeD3d4472fad1a6e2FFBb5B0E89835'}


In [19]:
pill_index = 5

pill_consumption_hash = contract_consumer.functions.consumeThePill(pill_index).transact({'from':accounts_list[3],'value': 1000000000000000000})
print(pillBookData('0x036b6384b5eca791c62761152d0c79bb0604c104a5fb6f4eb0703f3154bb3db0'))

{'Description': 5, 'strength(mg)': 10, 'imprints': 'PD 155 10', 'pregnancyCategory': 4, 'csaSchedule': 6, 'consumedFalse': True, 'consumedBy': '0x72280e1c8BbeD3d4472fad1a6e2FFBb5B0E89835'}


#### Taking Transaction Reciept

In [ ]:
bid_txn_receipt = web3.eth.waitForTransactionReceipt("0x367749f5b9e9785452ab76b6eec59e3e07a6ffa0452bab988aaf1cc35b73b03b")
print(bid_txn_receipt)

In [ ]:
tx_receipt = web3.eth.getTransactionReceipt('0x367749f5b9e9785452ab76b6eec59e3e07a6ffa0452bab988aaf1cc35b73b03b')
rich_logs = contract_manufacturer.events.PillCreated().processReceipt(tx_receipt)

In [ ]:
rich_logs[0]['args']

In [ ]:
rich_logs[0]['args']['description']

In [ ]:
rich_logs[0]['args']['message']